<a href="https://colab.research.google.com/github/Hyeok-Jun-Yoon/AI_Plus/blob/main/%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C_3%EC%A3%BC%EC%B0%A8_AI_2%EA%B8%B0_%EC%9C%A4%ED%98%81%EC%A4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본과제: DistilBERT로 뉴스 기사 분류 모델 학습하기

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver doe

DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hu

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

##[MYCODE] AG_News dataset 준비

def coolate_fn
*   truncate 하는 부분을 지우면 max_len 을 선언해도 의미가 없기 때문에 주석처리함.

*   padding 은 각각의 문장의 길이를 맞추어 주어야 하기 때문에 해줘야 함.




In [ ]:
ds = load_dataset("fancyzhx/ag_news")


def collate_fn(batch):
  #max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  tokenizer_encodings = tokenizer(texts, padding=True, return_tensors='pt')
  texts = tokenizer_encodings['input_ids']
  masks = tokenizer_encodings['attention_mask'] #[MYCODE]학습 시 attention_mask를 사용하기 위함

  labels = torch.LongTensor(labels)

  return texts, labels, masks

train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

train_loader 내용 확인

In [ ]:
for texts, labels, masks in train_loader:
  print(f"texts.shape : {texts.shape}")
  print(f"mask.shape : {masks.shape}")
  print(f"labels.shape : {labels.shape}")
  print(f"text :{texts}")
  print(f"mask : {masks}")
  print(f"labels : {labels}")
  break

texts.shape : torch.Size([64, 184])
mask.shape : torch.Size([64, 121])
labels.shape : torch.Size([64])
text :tensor([[  101,  2611, 13169,  ...,     0,     0,     0],
        [  101, 11990,  2000,  ...,     0,     0,     0],
        [  101,  4040,  8005,  ...,     0,     0,     0],
        ...,
        [  101, 10651,  1024,  ...,     0,     0,     0],
        [  101,  7861,  9307,  ...,     0,     0,     0],
        [  101,  7922,  2039,  ...,     0,     0,     0]])
mask : tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
labels : tensor([0, 0, 2, 3, 3, 0, 1, 2, 3, 3, 3, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 3, 0, 1,
        1, 2, 3, 2, 0, 1, 0, 2, 1, 0, 1, 2, 0, 0, 1, 0, 3, 2, 0, 1, 2, 0, 3, 1,
        2, 0, 0, 3, 1, 2, 2, 2, 0, 3, 3, 2, 3, 1, 3, 2])


## [MYCODE]


*   TEXT : 뉴스 기사의 Text
*   Labels : 각 텍스트의 카테고리



최종적으로 카테고리 갯수만큼 output 나와야함


In [ ]:
for batch in train_loader:
  texts, labels, masks = batch
  unique_categories = torch.unique(labels)

num_labels = len(unique_categories)

print(f"{unique_categories}  category num : {len(unique_categories)}")

tensor([0, 1, 2, 3])  category num : 4


DistilBERT model 선언

In [ ]:
encoder_model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
encoder_model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

## [MYCODE] Classifier output, loss function, accuracy function 변경

In [ ]:
from torch import nn


class TextClassifier(nn.Module):
  def __init__(self,encoder_model,num_labels):
    super().__init__()

    self.encoder = encoder_model
    self.classifier = nn.Linear(768, num_labels)

  def forward(self, x, masks):
    x = self.encoder(x, masks)['last_hidden_state']
    x = self.classifier(x[:, 0]) #[batch_size,hidden_size] , x[:,0] : [CLS] 토큰의 출력을 의미

    return x


model = TextClassifier(encoder_model,num_labels)

 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

In [ ]:
for param in model.encoder.parameters():
  param.requires_grad = False

## [MYCODE][LOG]학습 결과 REPORT

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss() #[MYCODE] CrossEntropyLoss 사용
optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels, masks = data
    inputs, labels, masks = inputs.to('cuda'), labels.to('cuda'), masks.to('cuda')

    preds = model(inputs, masks)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 665.8838135749102
Epoch   1 | Train Loss: 554.9611765667796
Epoch   2 | Train Loss: 537.4223141372204
Epoch   3 | Train Loss: 530.5255886614323
Epoch   4 | Train Loss: 525.170169133693
Epoch   5 | Train Loss: 520.5385025292635
Epoch   6 | Train Loss: 517.1208044961095
Epoch   7 | Train Loss: 518.3246390968561
Epoch   8 | Train Loss: 514.7026811391115
Epoch   9 | Train Loss: 512.4747037738562


In [ ]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels, masks = data
    inputs, labels, masks = inputs.to('cuda'), labels.to('cuda'), masks.to('cuda')

    preds = model(inputs, masks)
    preds = torch.argmax(preds, dim=-1)
    #preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.908 | Test acc: 0.904
